In [11]:
# from langchain_community.utilities import SQLDatabase


# db = SQLDatabase.from_uri("sqlite:///./data/houses_buy_test.db")
# print(db.dialect)
# print(db.get_usable_table_names())
# db.run("SELECT * FROM houses LIMIT 1;")

In [12]:
import dotenv
dotenv.load_dotenv()

True

In [13]:
# from langchain_community.agent_toolkits import create_sql_agent
# from langchain_openai import ChatOpenAI

# llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)
# agent_executor = create_sql_agent(llm, db=db, agent_type="openai-tools", verbose=True)

In [14]:
# agent_executor.invoke(
#     "Get me houses in maastricht that are lower than 250000 eur?"
# )

In [15]:
# agent_executor.invoke(
#     "I want to get a student friendly house in Nijmegen Explain How did you decide it is student friendly?"
# )

In [16]:
# from langchain.embeddings.openai import OpenAIEmbeddings
# from langchain_community.vectorstores import Chroma
# from langchain_community.document_loaders.csv_loader import CSVLoader


# embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

# loader = CSVLoader('./data/houses_buy_test.csv')
# houses = loader.load()
# houses

In [17]:
# persist_directory = './data/vecdb/'

# ## Here is the nmew embeddings being used
# embedding = OpenAIEmbeddings()

# db = Chroma.from_documents(documents=houses, 
#                             embedding=embedding,
#                             persist_directory=persist_directory,
#                             )

# db.persist()

In [18]:
from langchain_openai.embeddings import OpenAIEmbeddings
from langchain_community.vectorstores import Chroma

persist_directory = './data/vecdb/'

db = Chroma(persist_directory=persist_directory, 
            embedding_function=OpenAIEmbeddings())

In [19]:
query = "Student housing in Maastricht"
docs = db.similarity_search(query)
print(docs[0].page_content)

house_id: 43414815
url: https://www.funda.nl/koop/maastricht/appartement-43414815-aldenhofpark-1-a03/
city: maastricht
house_type: appartement
building_type: Bestaande bouw
price: 475000
price_m2: 6250.0
room: 3
bedroom: 2
bathroom: 1
living_area: 76
energy_label: na
zip: 6211
address: Aldenhofpark 1 a03
year_built: 1915
house_age: 109
descrip: MAASTRICHT (Jekerkwartier) -  In  het charmante  Jekerkwartier, direct  aan het stadspark van  Maastricht  op de 2e etage  gelegen instapklaar hoekappartement met  2 slaapkamers en  balkon  en een fantastisch  uitzicht op  de  kenmerkende  historische skyline met o.a. de torens van de Sint Janskerk en de Sint Servaeskerk aan het Vrijthof in Maastricht. Vanuit hier loop je zó het bruisende centrum in terwijl je niet direct de drukte van de binnenstad ervaart.  De ligging is werkelijk op een toplocatie, het onderhoud is uitstekend, de indeling zoals je het wenst: voorzien van een ruime living met uitstekende lichtinval mede dankzij de hoekerker en

In [20]:
from langchain import hub
from langchain_core.runnables import RunnablePassthrough
from langchain.schema import StrOutputParser
from langchain_openai import ChatOpenAI

retriever = db.as_retriever()

prompt = hub.pull("rlm/rag-prompt")
llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [21]:
from trulens_eval import TruChain, Tru
tru = Tru()
tru.reset_database()

🦑 Tru initialized with db url sqlite:///default.sqlite .
🛑 Secret keys may be written to the database. See the `database_redact_keys` option of Tru` to prevent this.


In [22]:
rag_chain.invoke("What are the avg price of hosues in MAastricht")

'The average price of houses in Maastricht ranges from €275,000 to €549,000, with the price per square meter varying from €2480.9 to €3873.2. The prices are influenced by factors such as location, size, and amenities of the properties.'

In [23]:
from trulens_eval.feedback.provider import OpenAI
from trulens_eval import Feedback
import numpy as np

# Initialize provider class
provider = OpenAI()

# select context to be used in feedback. the location of context is app specific.
from trulens_eval.app import App
context = App.select_context(rag_chain)

from trulens_eval.feedback import Groundedness
grounded = Groundedness(groundedness_provider=OpenAI())
# Define a groundedness feedback function
f_groundedness = (
    Feedback(grounded.groundedness_measure_with_cot_reasons)
    .on(context.collect()) # collect context chunks into a list
    .on_output()
    .aggregate(grounded.grounded_statements_aggregator)
)

# Question/answer relevance between overall question and answer.
f_answer_relevance = (
    Feedback(provider.relevance)
    .on_input_output()
)
# Question/statement relevance between question and each context chunk.
f_context_relevance = (
    Feedback(provider.context_relevance_with_cot_reasons)
    .on_input()
    .on(context)
    .aggregate(np.mean)
)

✅ In groundedness_measure_with_cot_reasons, input source will be set to __record__.app.first.steps.context.first.get_relevant_documents.rets.collect() .
✅ In groundedness_measure_with_cot_reasons, input statement will be set to __record__.main_output or `Select.RecordOutput` .
✅ In relevance, input prompt will be set to __record__.main_input or `Select.RecordInput` .
✅ In relevance, input response will be set to __record__.main_output or `Select.RecordOutput` .
✅ In context_relevance_with_cot_reasons, input question will be set to __record__.main_input or `Select.RecordInput` .
✅ In context_relevance_with_cot_reasons, input context will be set to __record__.app.first.steps.context.first.get_relevant_documents.rets .


In [24]:
tru_recorder = TruChain(rag_chain,
    app_id='Chain1_ChatApplication',
    feedbacks=[f_answer_relevance, f_context_relevance, f_groundedness])

In [25]:
response, tru_record = tru_recorder.with_record(rag_chain.invoke, "What are the houses around AMsterdam that are pet friendly")

In [26]:
json_like = tru_record.layout_calls_as_app()
json_like

Munch({'record_id': 'record_hash_d94111b5f6db35fb32bf1505c58efccc', 'app_id': 'Chain1_ChatApplication', 'cost': {'n_requests': 2, 'n_successful_requests': 2, 'n_classes': 0, 'n_tokens': 8485, 'n_stream_chunks': 0, 'n_prompt_tokens': 8480, 'n_completion_tokens': 5, 'cost': 0.012713499999999999}, 'perf': {'start_time': '2024-03-29T12:34:21.627954', 'end_time': '2024-03-29T12:34:24.155055'}, 'ts': '2024-03-29T12:34:24.155414', 'tags': '-', 'meta': None, 'main_input': 'What are the houses around AMsterdam that are pet friendly', 'main_output': "I don't know.", 'main_error': None, 'calls': [{'stack': [{'path': 'app', 'method': {'obj': {'cls': {'name': 'RunnableSequence', 'module': {'package_name': 'langchain_core.runnables', 'module_name': 'langchain_core.runnables.base'}, 'bases': None}, 'id': 139679999504512, 'init_bindings': None}, 'name': 'invoke'}}, {'path': 'app.first', 'method': {'obj': {'cls': {'name': 'RunnableParallel', 'module': {'package_name': 'langchain_core.runnables', 'modul

In [27]:
from ipytree import Tree, Node

def display_call_stack(data):
    tree = Tree()
    tree.add_node(Node('Record ID: {}'.format(data['record_id'])))
    tree.add_node(Node('App ID: {}'.format(data['app_id'])))
    tree.add_node(Node('Cost: {}'.format(data['cost'])))
    tree.add_node(Node('Performance: {}'.format(data['perf'])))
    tree.add_node(Node('Timestamp: {}'.format(data['ts'])))
    tree.add_node(Node('Tags: {}'.format(data['tags'])))
    tree.add_node(Node('Main Input: {}'.format(data['main_input'])))
    tree.add_node(Node('Main Output: {}'.format(data['main_output'])))
    tree.add_node(Node('Main Error: {}'.format(data['main_error'])))
    
    calls_node = Node('Calls')
    tree.add_node(calls_node)
    
    for call in data['calls']:
        call_node = Node('Call')
        calls_node.add_node(call_node)
        
        for step in call['stack']:
            step_node = Node('Step: {}'.format(step['path']))
            call_node.add_node(step_node)
            if 'expanded' in step:
                expanded_node = Node('Expanded')
                step_node.add_node(expanded_node)
                for expanded_step in step['expanded']:
                    expanded_step_node = Node('Step: {}'.format(expanded_step['path']))
                    expanded_node.add_node(expanded_step_node)
    
    return tree

# Usage
tree = display_call_stack(json_like)
tree

Groundedness per statement in source:   0%|          | 0/1 [00:00<?, ?it/s]

Tree(nodes=(Node(name='Record ID: record_hash_d94111b5f6db35fb32bf1505c58efccc'), Node(name='App ID: Chain1_Ch…

In [ ]:
with tru_recorder as recording:
    llm_response = rag_chain.invoke("Whata re the houses in Maastricht that are closer to the university?")

display(llm_response)

'The average price of houses in Maastricht ranges from €275,000 to €549,000, with an average price per square meter of €2480.9 to €3873.2. The prices vary depending on the specific location, size, and type of the property.'

Groundedness per statement in source:   0%|          | 0/2 [00:00<?, ?it/s]

In [35]:
# The record of the app invocation can be retrieved from the `recording`:

rec = recording.get() # use .get if only one record
# recs = recording.records # use .records if multiple

display(rec)

Record(record_id='record_hash_e4a251dbadec0d5fb1cbabc5e6955929', app_id='Chain1_ChatApplication', cost=Cost(n_requests=2, n_successful_requests=2, n_classes=0, n_tokens=8007, n_stream_chunks=0, n_prompt_tokens=7948, n_completion_tokens=59, cost=0.012022), perf=Perf(start_time=datetime.datetime(2024, 3, 29, 12, 35, 58, 768652), end_time=datetime.datetime(2024, 3, 29, 12, 36, 1, 916872)), ts=datetime.datetime(2024, 3, 29, 12, 36, 1, 917005), tags='-', meta=None, main_input='What are the avg price of hosues in MAastricht', main_output='The average price of houses in Maastricht ranges from €275,000 to €549,000, with an average price per square meter of €2480.9 to €3873.2. The prices vary depending on the specific location, size, and type of the property.', main_error=None, calls=[RecordAppCall(stack=[RecordAppCallMethod(path=Lens().app, method=Method(obj=Obj(cls=langchain_core.runnables.base.RunnableSequence, id=139679999504512, init_bindings=None), name='invoke')), RecordAppCallMethod(pat

In [36]:
# The results of the feedback functions can be rertireved from
# `Record.feedback_results` or using the `wait_for_feedback_result` method. The
# results if retrieved directly are `Future` instances (see
# `concurrent.futures`). You can use `as_completed` to wait until they have
# finished evaluating or use the utility method:

for feedback, feedback_result in rec.wait_for_feedback_results().items():
    print(feedback.name, feedback_result.result)

# See more about wait_for_feedback_results:
# help(rec.wait_for_feedback_results)

relevance 0.8
context_relevance_with_cot_reasons 0.8
groundedness_measure_with_cot_reasons 0.0


In [37]:
records, feedback = tru.get_records_and_feedback(app_ids=["Chain1_ChatApplication"])

records.head()

,app_id,app_json,type,record_id,input,output,tags,record_json,cost_json,perf_json,ts,relevance,groundedness_measure_with_cot_reasons,context_relevance_with_cot_reasons,relevance_calls,groundedness_measure_with_cot_reasons_calls,context_relevance_with_cot_reasons_calls,latency,total_tokens,total_cost
0,Chain1_ChatApplication,"{""tru_class_info"": {""name"": ""TruChain"", ""modul...",RunnableSequence(langchain_core.runnables.base),record_hash_d94111b5f6db35fb32bf1505c58efccc,"""What are the houses around AMsterdam that are...","""I don't know.""",-,"{""record_id"": ""record_hash_d94111b5f6db35fb32b...","{""n_requests"": 2, ""n_successful_requests"": 2, ...","{""start_time"": ""2024-03-29T12:34:21.627954"", ""...",2024-03-29T12:34:24.155414,1.0,0.0,0.4,[{'args': {'prompt': 'What are the houses arou...,"[{'args': {'source': [[{'page_content': ""house...",[{'args': {'question': 'What are the houses ar...,2,8485,0.012713
1,Chain1_ChatApplication,"{""tru_class_info"": {""name"": ""TruChain"", ""modul...",RunnableSequence(langchain_core.runnables.base),record_hash_682f63d68c29282b2cbb85493105921a,"""What is Task Decomposition?""","""Task Decomposition is the process of breaking...",-,"{""record_id"": ""record_hash_682f63d68c29282b2cb...","{""n_requests"": 2, ""n_successful_requests"": 2, ...","{""start_time"": ""2024-03-29T12:34:25.773902"", ""...",2024-03-29T12:34:30.036455,1.0,0.0,0.2,[{'args': {'prompt': 'What is Task Decompositi...,"[{'args': {'source': [[{'page_content': ""house...",[{'args': {'question': 'What is Task Decomposi...,4,7985,0.011998
2,Chain1_ChatApplication,"{""tru_class_info"": {""name"": ""TruChain"", ""modul...",RunnableSequence(langchain_core.runnables.base),record_hash_e4a251dbadec0d5fb1cbabc5e6955929,"""What are the avg price of hosues in MAastricht""","""The average price of houses in Maastricht ran...",-,"{""record_id"": ""record_hash_e4a251dbadec0d5fb1c...","{""n_requests"": 2, ""n_successful_requests"": 2, ...","{""start_time"": ""2024-03-29T12:35:58.768652"", ""...",2024-03-29T12:36:01.917005,0.8,0.0,0.8,[{'args': {'prompt': 'What are the avg price o...,[{'args': {'source': [[{'page_content': 'house...,[{'args': {'question': 'What are the avg price...,3,8007,0.012022


In [38]:
tru.run_dashboard() # open a local streamlit app to explore

# tru.stop_dashboard() # stop if needed

Starting dashboard ...


Accordion(children=(VBox(children=(VBox(children=(Label(value='STDOUT'), Output())), VBox(children=(Label(valu…